# Import package

In [ ]:
import os
import torch
import prism
from prism import train
print(prism.__version__)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
torch.cuda.set_device(0) 

/opt/conda/envs/PRISM/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.2.0


# Set parameters and data path

In [2]:
arg_path = "./param.yaml"
Expression_data_path = "../demo_data/A549/ExpressionData.csv"
Genescore_data_path = "../demo_data/A549/GeneScoreData.csv"
label_path = "../demo_data/A549/refNetwork.csv"

## import args and show

In [3]:
args = train.ImportArgs(arg_path)
print(args)

{'flag': False, 'cuda': True, 'seed': 2222, 'epoch': 2000, 'lr': 0.0003, 'weight_decay': 0.001, 'decayrate': 0.97, 'hidden': 256, 'dropout': 0.1, 'ns': 64, 'alpha': 0.35, 'pairing': True, 'device': device(type='cuda', index=0)}


# The implementation of PRISM

## For validation with baselines (split GRN into training and testing)

In [4]:
## Instance
A549_prism = train.Trainer(args, Expression_data_path, Genescore_data_path, label_path)

In [5]:
## training
A549_prism.train()

On validation epoch 100: RNA recon loss 22598.9375, GRN recon loss 8678.7646484375
Validation Accuracy: 0.9902761578559875
On validation epoch 200: RNA recon loss 11317.5625, GRN recon loss 8396.5615234375
Validation Accuracy: 0.9902761578559875
On validation epoch 300: RNA recon loss 10894.375, GRN recon loss 6152.267578125
Validation Accuracy: 0.9902244806289673
On validation epoch 400: RNA recon loss 10476.3828125, GRN recon loss 4012.78662109375
Validation Accuracy: 0.9896175861358643
On validation epoch 500: RNA recon loss 10250.734375, GRN recon loss 3740.632568359375
Validation Accuracy: 0.9903084635734558
On validation epoch 600: RNA recon loss 10228.484375, GRN recon loss 3593.27294921875
Validation Accuracy: 0.9902761578559875
On validation epoch 700: RNA recon loss 10201.234375, GRN recon loss 3520.259521484375
Validation Accuracy: 0.990502119064331
On validation epoch 800: RNA recon loss 10192.0546875, GRN recon loss 3482.94091796875
Validation Accuracy: 0.9907022714614868


In [6]:
## Saving model
A549_prism.save_model()

Model saved at prism_model.pth


In [8]:
## Get the Predicted GRN
Reconstruct_GRN = A549_prism.Get_GRN()
print(Reconstruct_GRN)

         Gene1      Gene2      Prob  Pred_Label
0          SP1     TRIM47  0.275982         0.0
1          SP1       SDC4  0.249377         0.0
2         JUNB     ZNF768  0.289184         0.0
3          MAX       OSR1  0.411708         0.0
4         ELF1     MAD2L2  0.272297         0.0
...        ...        ...       ...         ...
154872   FEZF1     SKIDA1  0.000004         0.0
154873   NPY1R  LINC00376  0.000011         0.0
154874   TFPI2       DAD1  0.000007         0.0
154875    CIB1   TMEM200B  0.000004         0.0
154876  GALNT3   HM13-AS1  0.000010         0.0

[154877 rows x 4 columns]


## For in-reality use (all prior GRNs are used to training)

In [9]:
A549_prism = train.Trainer_allprior(args, Expression_data_path, Genescore_data_path, label_path)

In [7]:
A549_prism.train()

On tranining epoch 100: RNA recon loss 11250.6796875, GRN recon loss 18970.126953125
On tranining epoch 200: RNA recon loss 10330.5078125, GRN recon loss 3971.816650390625
On tranining epoch 300: RNA recon loss 10162.640625, GRN recon loss 3040.315673828125
On tranining epoch 400: RNA recon loss 10183.3125, GRN recon loss 2917.8759765625
On tranining epoch 500: RNA recon loss 10145.5546875, GRN recon loss 2868.9375
On tranining epoch 600: RNA recon loss 10157.8828125, GRN recon loss 2842.752685546875
On tranining epoch 700: RNA recon loss 10177.609375, GRN recon loss 2783.949951171875
On tranining epoch 800: RNA recon loss 10181.625, GRN recon loss 2777.13818359375
On tranining epoch 900: RNA recon loss 10189.7890625, GRN recon loss 2732.412353515625
On tranining epoch 1000: RNA recon loss 10194.140625, GRN recon loss 2722.75341796875
On tranining epoch 1100: RNA recon loss 10193.8984375, GRN recon loss 2717.31494140625
On tranining epoch 1200: RNA recon loss 10219.0546875, GRN recon l

In [14]:
Reconstruct_GRN = A549_prism.Get_GRN()
print(Reconstruct_GRN[Reconstruct_GRN['Pred_Label']==1])

           Gene1    Gene2      Prob  Pred_Label
40          PHF8    NPY1R  0.961837         1.0
42         MYBL2     TAC4  0.710018         1.0
97         MYOCD    TRAT1  0.973083         1.0
107      BHLHE40    MOCS1  0.836999         1.0
190        SMAD3   TUBB4A  0.877208         1.0
...          ...      ...       ...         ...
1518556      EHF    NOTUM  0.514836         1.0
1518566    NPAS2   KCTD14  0.514870         1.0
1518573     E2F6  ABHD17A  0.977294         1.0
1518582   HMGXB4    DDX11  0.634113         1.0
1518608     BRD2   MTMR11  0.969103         1.0

[82952 rows x 4 columns]
